In [1]:
import pandas as pd
import config, requests, time

In [2]:
# https://www.alphavantage.co/documentation/

In [3]:
BASE_URL = 'https://www.alphavantage.co/query?function='
function = 'TIME_SERIES_DAILY' #returns daily data vs. intraday data
symbol = '' #stock ticker - single value allowed only
outputsize = 'full' #full or compact. compact returns 100 data points

In [4]:
request_URL = BASE_URL + \
f'{function}&symbol={symbol}&outputsize={outputsize}&apikey={config.ALPHA_VANTAGE_API}'

In [5]:
# create new (empty) dataframe
df = pd.DataFrame()

In [6]:
# list tickers to get data for
symbols = ['MSFT', 'AMD', 'TSLA', 'JNJ', 'REGN', 'GILD']

# loop through ticker list, create dataframe for each, and join to dataframe created earlier
for symbol in symbols:
#     set request URL for GET request
    request_URL = BASE_URL + \
                f'{function}&symbol={symbol}&outputsize={outputsize}\
&apikey={config.ALPHA_VANTAGE_API}'
#     send GET request
    r = requests.get(request_URL)
#     create dataframe from GET response (transpose dataframe, as response flips
#     rows and columns (dates are in columns instead of rows)
    _ = pd.DataFrame(r.json()['Time Series (Daily)']).transpose()
#     rename columns to include ticker
    columns = [
        f'{symbol}_open',
        f'{symbol}_high',
        f'{symbol}_low',
        f'{symbol}_close',
        f'{symbol}_volume'
    ]
    _.columns = columns
#     join dataframe to "master" dataframe
    df = df.join(_, how='outer')
    time.sleep(15) #sleeping due to call limit on api (5 calls per minute)

In [7]:
df.head()

,MSFT_open,MSFT_high,MSFT_low,MSFT_close,MSFT_volume,AMD_open,AMD_high,AMD_low,AMD_close,AMD_volume,...,REGN_open,REGN_high,REGN_low,REGN_close,REGN_volume,GILD_open,GILD_high,GILD_low,GILD_close,GILD_volume
2000-07-06,78.8700,81.6900,78.5000,80.9400,23936600,74.5000,77.8800,73.0000,77.5000,5493700,...,28.5600,29.4400,26.5600,28.9400,157500,71.0600,73.5000,70.8800,73.1300,493900
2000-07-07,81.2700,82.8700,80.5600,82.0000,27076600,77.7500,83.4400,77.5600,82.0000,4503500,...,29.1900,33.1900,28.2500,29.6300,346500,74.0000,81.7500,74.0000,80.3100,1233100
2000-07-10,80.6900,81.3100,79.3700,79.4400,26344900,86.2500,87.6300,83.7500,85.0000,5099900,...,28.8100,30.0000,28.6300,29.7500,275400,80.0000,81.4400,75.0000,79.3800,554400
2000-07-11,78.8100,80.5000,78.5000,79.1200,18582400,84.2500,86.1900,81.5600,83.5000,3811900,...,29.7500,31.7500,29.6300,31.3800,312800,80.0600,83.8800,79.1900,83.0000,553900
2000-07-12,78.2500,81.0000,77.1900,80.3100,29185100,84.0000,85.1900,82.7500,84.3100,3263500,...,32.0000,32.2500,29.6300,29.6900,284500,82.8800,90.1900,82.3800,89.3800,992900


In [8]:
df.describe()

,MSFT_open,MSFT_high,MSFT_low,MSFT_close,MSFT_volume,AMD_open,AMD_high,AMD_low,AMD_close,AMD_volume,...,REGN_open,REGN_high,REGN_low,REGN_close,REGN_volume,GILD_open,GILD_high,GILD_low,GILD_close,GILD_volume
count,5032,5032,5032,5032,5032,5032,5032,5032,5032,5032,...,5032,5032,5032,5032,5032,5032,5032,5032,5032,5032
unique,3123,3150,3150,3193,5023,2241,2367,2304,2282,5007,...,3848,3884,3882,3923,4534,3429,3470,3523,3500,4976
top,27.0800,26.0000,25.5000,27.2500,34904200,4.0300,2.7000,3.6000,3.6900,15527600,...,24.5000,15.0000,26.0000,13.0000,504100,46.0000,46.0000,45.0000,36.5000,3337300
freq,11,14,16,12,2,13,16,14,15,2,...,8,10,7,9,4,9,9,8,8,3


In [9]:
df.isna().sum()

MSFT_open         0
MSFT_high         0
MSFT_low          0
MSFT_close        0
MSFT_volume       0
AMD_open          0
AMD_high          0
AMD_low           0
AMD_close         0
AMD_volume        0
TSLA_open      2509
TSLA_high      2509
TSLA_low       2509
TSLA_close     2509
TSLA_volume    2509
JNJ_open          0
JNJ_high          0
JNJ_low           0
JNJ_close         0
JNJ_volume        0
REGN_open         0
REGN_high         0
REGN_low          0
REGN_close        0
REGN_volume       0
GILD_open         0
GILD_high         0
GILD_low          0
GILD_close        0
GILD_volume       0
dtype: int64

In [10]:
# Clean up dataframe

# remove N/A's
df.dropna(inplace=True)

# remove (potentially) unused columns
df = df[df.columns.drop(list(column for column in df.columns if column.lower()[-7:] == '_volume'))]

# sort index in ascending order
df.sort_index(ascending=True, inplace=True)

In [11]:
df.describe()

,MSFT_open,MSFT_high,MSFT_low,MSFT_close,AMD_open,AMD_high,AMD_low,AMD_close,TSLA_open,TSLA_high,...,JNJ_low,JNJ_close,REGN_open,REGN_high,REGN_low,REGN_close,GILD_open,GILD_high,GILD_low,GILD_close
count,2523,2523,2523,2523,2523,2523,2523,2523,2523,2523,...,2523,2523,2523,2523,2523,2523,2523,2523,2523,2523
unique,2109,2110,2119,2132,1363,1434,1370,1370,2244,2265,...,2127,2138,2405,2416,2418,2431,2029,2119,2097,2062
top,27.9300,26.0000,25.5000,25.8100,2.6900,2.7000,3.6000,3.6900,28.0000,28.0000,...,64.8400,103.2200,367.0000,410.0000,178.0000,390.2800,70.0000,75.2500,70.7200,72.7800
freq,5,5,5,6,11,16,13,14,7,7,...,5,5,5,5,4,3,7,6,5,6


In [12]:
df.to_csv('stocks_history.csv')